In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [7]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/uas/atl4/'
dir_hist_z23 = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/z20/atl3/'
dir_hist_z23_2 = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/z20/atl4/'

dir_rcp8_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/uas/atl4/'
dir_rcp8_z23 = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/z20/atl3/'
dir_rcp8_z23_2 = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/z20/atl4/'

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099

def is_jja(month):
    return (month >= 6) & (month <= 8)

now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [3]:
model = ['ACCESS1-0', 'ACCESS1-3','CMCC-CESM',
         'CMCC-CMS','CMCC-CM','CNRM-CM5','CSIRO-Mk3-6-0',
        'GFDL-CM3', 'GFDL-ESM2M','GISS-E2-H-CC','GISS-E2-H',
        'GISS-E2-R-CC','GISS-E2-R','HadGEM2-CC','IPSL-CM5A-LR',
        'IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC-ESM-CHEM','MIROC-ESM',
        'MIROC5','MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M']

# Historical

In [5]:


alpha_bf2_hist_jja = np.ones((len(model)))
r_bf2_hist_jja = np.ones((len(model)))
std_bf2_hist_jja = np.ones((len(model)))
p_value_bf2_hist_jja = np.ones((len(model)))

alpha_bf2_hist_monthly = np.ones((len(model),12))
r_bf2_hist_monthly = np.ones((len(model),12))
std_bf2_hist_monthly = np.ones((len(model),12))
p_value_bf2_hist_monthly = np.ones((len(model),12))



model_id = []

for i in range(0,len(model)):
    

    
    uas = xr.open_dataset(dir_hist_uas+'uas_Amon_'+model[i]+'_historical_r1i1p1_187001-200012_1deg_atl4.nc')


    try:

        uas = uas.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        uas['time'] = uas.indexes['time'].to_datetimeindex()
        uas = uas.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    uas = uas.__xarray_dataarray_variable__.chunk(chunks=None)
    uas = xsf.detrend(uas[:],dim='time',type='linear')
    uas = uas.load()
    uasa_atl4, _ = Atools.ano_norm_t(uas.mean(dim='lon').mean(dim='lat'))



    
    z23_1 = xr.open_dataset(dir_hist_z23+'z20_Omon_'+model[i]+'_historical_r1i1p1_187001-200012_1deg_atl3.nc')
    z23_2 = xr.open_dataset(dir_hist_z23_2+'z20_Omon_'+model[i]+'_historical_r1i1p1_187001-200012_1deg_atl4.nc')
    
    print(model[i])
    

    try:

        z23_1 = z23_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
        z23_2 = z23_2.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        z23_1['time'] = z23_1.indexes['time'].to_datetimeindex()
        z23_1 = z23_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
        z23_2['time'] = z23_2.indexes['time'].to_datetimeindex()
        z23_2 = z23_2.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
     
    try:
        thermoslope =     z23_1.mean(dim='lon').mean(dim='lat') - z23_2.mean(dim='lon').mean(dim='lat')
        
        
    except ValueError:
        thermoslope =     z23_1.mean(dim='x').mean(dim='y') - z23_2.mean(dim='x').mean(dim='y')
        
        
    thermoslope_tmp = thermoslope.z20.chunk(chunks=None)
    thermoslope_tmp2 = xsf.detrend(thermoslope_tmp[:],dim='time',type='linear')
    thermoslope_tmp2 = thermoslope_tmp2.load()
    thermoslopea, _ = Atools.ano_norm_t(thermoslope_tmp2)
    thermoslopea_atl3 = thermoslopea

    uasa_atl4_jja = uasa_atl4.sel(time=is_jja(uasa_atl4['time.month']))
    thermoslopea_atl3_jja = thermoslopea_atl3.sel(time=is_jja(thermoslopea_atl3['time.month']))

    alpha_bf2_hist_jja[i], _, r_bf2_hist_jja[i], p_value_bf2_hist_jja[i], std_bf2_hist_jja[i] = stats.linregress(
    uasa_atl4_jja[:],thermoslopea_atl3_jja[:])
    
    for j in range(0,12):
        alpha_bf2_hist_monthly[i,j], _, r_bf2_hist_monthly[i,j], p_value_bf2_hist_monthly[i,j], std_bf2_hist_monthly[i,j] = stats.linregress(
    uasa_atl4.sel(time=uasa_atl4['time.month'] == j+1), thermoslopea_atl3.sel(time=thermoslopea_atl3['time.month'] == j+1))
     

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS1-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS1-3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


...

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


MRI-CGCM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


NorESM1-M


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


In [8]:
bf2_hist_jja  = xr.Dataset({'alpha': (['product'],alpha_bf2_hist_jja),
                       'r': (['product'],r_bf2_hist_jja),
                       'p_value': (['product'],p_value_bf2_hist_jja),
                       'std': (['product'],std_bf2_hist_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF2',
                                    'long_name': 'Second Bjerknes feedback component',
                                    'units': 'm/(m/s)',
                                    'model': 'CMIP5',
                                    'Scenario': 'HIST',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})



In [9]:
#bf2_hist_monthly.to_netcdf('/Volumes/Arthur_disk2/Data/CMIP6/tmp_data/bf2_hist_monthly_CMIP5_z20.nc')
bf2_hist_jja.to_netcdf(path_data+'bf2_hist_jja_CMIP5_z20.nc')

# rcp85

In [10]:


alpha_bf2_rcp8_jja = np.ones((len(model)))
r_bf2_rcp8_jja = np.ones((len(model)))
std_bf2_rcp8_jja = np.ones((len(model)))
p_value_bf2_rcp8_jja = np.ones((len(model)))

alpha_bf2_rcp8_monthly = np.ones((len(model),12))
r_bf2_rcp8_monthly = np.ones((len(model),12))
std_bf2_rcp8_monthly = np.ones((len(model),12))
p_value_bf2_rcp8_monthly = np.ones((len(model),12))



for i in range(0,len(model)):
    
    uas = xr.open_dataset(dir_rcp8_uas+'uas_Amon_'+model[i]+'_rcp85_r1i1p1_201001-210012_1deg_atl4.nc')

    try:

        uas = uas.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        uas['time'] = uas.indexes['time'].to_datetimeindex()
        uas = uas.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
    uas = uas.__xarray_dataarray_variable__.chunk(chunks=None)
    uas = xsf.detrend(uas[:],dim='time',type='linear')
    uas = uas.load()
    uasa_atl4, _ = Atools.ano_norm_t(uas.mean(dim='lon').mean(dim='lat'))


    z23_1 = xr.open_dataset(dir_rcp8_z23+'z20_Omon_'+model[i]+'_rcp85_r1i1p1_201501-209912_1deg_atl3.nc')

    z23_2 = xr.open_dataset(dir_rcp8_z23_2+'z20_Omon_'+model[i]+'_rcp85_r1i1p1_201501-209912_1deg_atl4.nc')

    print(model[i])

    try:

        z23_1 = z23_1.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        z23_2 = z23_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        z23_1['time'] = z23_1.indexes['time'].to_datetimeindex()
        z23_1 = z23_1.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
        z23_2['time'] = z23_2.indexes['time'].to_datetimeindex()
        z23_2 = z23_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
        
    try:
        thermoslope =     z23_1.mean(dim='lon').mean(dim='lat') - z23_2.mean(dim='lon').mean(dim='lat')
        
        
    except ValueError:
        thermoslope =     z23_1.mean(dim='x').mean(dim='y') - z23_2.mean(dim='x').mean(dim='y') 
        
    thermoslope = thermoslope.z20.chunk(chunks=None)
    thermoslope = xsf.detrend(thermoslope,dim='time',type='linear')
    thermoslope = thermoslope.load()
    thermoslopea, _ = Atools.ano_norm_t(thermoslope)
    thermoslopea_atl3 = thermoslopea

    uasa_atl4_jja = uasa_atl4.sel(time=is_jja(uasa_atl4['time.month']))
    thermoslopea_atl3_jja = thermoslopea_atl3.sel(time=is_jja(thermoslopea_atl3['time.month']))

    alpha_bf2_rcp8_jja[i], _, r_bf2_rcp8_jja[i], p_value_bf2_rcp8_jja[i], std_bf2_rcp8_jja[i] = stats.linregress(
    uasa_atl4_jja[:],thermoslopea_atl3_jja[:])
    
    
    for j in range(0,12):
        alpha_bf2_rcp8_monthly[i,j], _, r_bf2_rcp8_monthly[i,j], p_value_bf2_rcp8_monthly[i,j], std_bf2_rcp8_monthly[i,j] = stats.linregress(
    uasa_atl4.sel(time=uasa_atl4['time.month'] == j+1), thermoslopea_atl3.sel(time=thermoslopea_atl3['time.month'] == j+1))
     

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS1-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS1-3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


...

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


MRI-CGCM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


NorESM1-M


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


In [11]:
bf2_rcp8_jja  = xr.Dataset({'alpha': (['product'],alpha_bf2_rcp8_jja),
                       'r': (['product'],r_bf2_rcp8_jja),
                       'p_value': (['product'],p_value_bf2_rcp8_jja),
                       'std': (['product'],std_bf2_rcp8_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF2',
                                    'long_name': 'Second Bjerknes feedback component',
                                    'units': 'm/(m/s)',
                                    'model': 'CMIP5',
                                    'Scenario': 'RCP85',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


In [12]:

bf2_rcp8_jja.to_netcdf(path_data+'bf2_ssp5_jja_CMIP5_z20.nc')